In [1]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from scipy.signal import medfilt
from tqdm.notebook import tqdm

In [2]:
#Range time data
def range_time(IQ_data, overlap = 1): 
    n_rd_history = 256
    frame = []
    frames = []
    overlap_count = 0
    check = 0
    for iqini in IQ_data:
        if len(frame)<n_rd_history:
            frame.append(iqini)
        else:  
            if check == 0:
                frame1 = np.array(frame)
                frames.append(np.copy(frame1))
                check = 1

            if overlap_count < overlap:
                frame.append(iqini)
                overlap_count = overlap_count+1

            if overlap_count == overlap:
                frame = frame[overlap_count::]
                frame1 = np.array(frame)
                frames.append(np.copy(frame1))
                overlap_count = 0
    return np.stack(frames)

import scipy.fftpack as fft
#Range frequency data
def range_frequency(datas):
    Range_frequency_frame = []
    for data in datas:
        jitter = 1e-10
        # noise_threshold = -65
        dB = True
        #rd_history = np.hanning(n_rd_history)[:, None] * np.array(data)
        # Range-Doppler
        rd = fft.fft(data, axis=0)
        rd = fft.fftshift(rd, axes=0)
        rd = abs(rd)
        if dB:
            rd = 20 * np.log10(rd+jitter)
            # rd[rd < noise_threshold] = noise_threshold
        Range_frequency_frame.append(rd)
    return np.stack(Range_frequency_frame)

def get_time_img(data):
    # output = np.load(f'{data_path}{data_id}')
    output = np.abs(range_time(data, overlap=256))
    output = output.reshape(-1,56)
    output -= output.mean(axis=0)
    return output.reshape(-1,56)

def get_freq_img(data):
    # output = np.load(f'{data_path}{data_id}')
    output = range_time(data, overlap=256)
    # output -= output.mean(axis=0)
    output = range_frequency(output)
    return output.reshape(-1,56)

In [3]:
def save_fig(X, filename, mode):
    fig, ax = plt.subplots(figsize=(10, 10)) # set figure size
    # ax.imshow(np.abs(X)[:,10:], aspect='auto', cmap='gray') # slice 
    ax.imshow(np.abs(X), aspect='auto', cmap='gray') # slice 
    ax.set_axis_off()

    if mode == 'Train':
        fig.savefig(f'freq_img_gray_bgsup/train/{filename}.jpg', bbox_inches='tight', pad_inches=0)
    elif mode == 'Test':
        fig.savefig(f'freq_img_gray_bgsup/test/{filename}.jpg', bbox_inches='tight', pad_inches=0)
    plt.close()

In [4]:
def save_dataset():
    
    if not os.path.exists('freq_img_gray_bgsup/train'):
        os.mkdir('freq_img_gray_bgsup/train')
        
    if not os.path.exists('freq_img_gray_bgsup/test'):
        os.mkdir('freq_img_gray_bgsup/test')
    
    # Setting Train, test path
    train_list = glob.glob('train/train/*.npy')
    test_list = glob.glob('test/test/*.npy')
    annotations = pd.read_csv('annotations.csv')
    
    # Import Train data
    X_train = []
    y_train = []
    
    for i in tqdm(train_list):        
        z = get_freq_img(np.load(i))
        # z -= z.mean(axis=0)
        label = annotations[annotations['id'] == i.replace('train/train/','').replace('.npy','')]['class'].values
        
        # check if shape is all the same
        if z.shape != (2560,56):
            print(i)
            print("Outlier shape: ", z.shape)
            print('=='*5)
        else:    
            # X_train.append(z)
            # y_train.append(label)

            filename = i.replace('train/train/','').replace('.npy','')
            mode = 'Train'
            save_fig(z, filename, mode)
        
    print('=='*5, 'Train Finish')
    
    
    # Import test data
    X_test = []
    for i in tqdm(test_list):

        z = get_freq_img(np.load(i))
        # z -= z.mean(axis=0)
        # X_test.append(z)

        filename = i.replace('test/test/','').replace('.npy','')
        mode = 'Test'
        save_fig(z, filename, mode)

    print('=='*5, 'Test Finish')

In [5]:
save_dataset()

  0%|          | 0/647 [00:00<?, ?it/s]

train/train/8dbc522d-23b7-4e25-936d-54f749f96e25.npy
Outlier shape:  (7680, 56)
========== Train Finish


  0%|          | 0/164 [00:00<?, ?it/s]

========== Test Finish
